In [0]:
#pip install  -q -U llama-index PyPDF2
#dbutils.library.restartPython()

In [0]:
pdf_path ="/Volumes/workspace/default/test_data"

In [0]:

pdf_path = "/Volumes/workspace/default/test_data"
#Deine the output table name
table_name = "default.test_data_pdf_index"

# Read all ODF files in the folder
df = (
    spark.read.format("binaryFile")
    .option("recursiveFileLookup", "true")
    .load(pdf_path)
)

# Write the Dataframe as a table (for file indexing/logging)
df.write.mode("overwrite").saveAsTable(table_name)

display(df)


In [0]:
df.limit(20).select("path", "length")


In [0]:
def parse_bytes_pypdf(pdf_bytes):
    from PyPDF2 import PdfReader
    import io
    try:
        reader = PdfReader(io.BytesIO(pdf_bytes))
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
        return text.strip()
    except Exception as e:
        print(f"Error parsing PDF: {e}")
        return None

try:   
    with open(f"{pdf_path}/", mode="rb") as pdf_file:
        pdf_bytes = pdf.read()
        print("PDF read succesfully")
        doc = parse_bytes_pypdf(pdf.read())
        if doc: 
            print(doc)
        print("File not found")
except Exception as e:
        print(f"An error occurred: {e}")

In [0]:
#pip install pytesseract pdf2image

In [0]:
def parse_bytes_pypdf(pdf_bytes):
    from PyPDF2 import PdfReader
    import io
    try:
        reader = PdfReader(io.BytesIO(pdf_bytes))
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
        return text.strip()
    except Exception as e:
        print(f"Error parsing PDF: {e}")
        return None

pdf_path = "/Volumes/workspace/default/test_data"

try:   
    with open(f"{pdf_path}/Fast_Resampling_three_dimensional_points_clouds.pdf", mode="rb") as pdf_file:
        pdf_bytes = pdf_file.read()
        print("PDF read successfully")
        doc = parse_bytes_pypdf(pdf_bytes)
        if doc: 
            print(doc)
        else:
            print("No text extracted from PDF")
except Exception as e:
    print(f"An error occurred: {e}")

In [0]:
%pip install transformers

In [0]:
import io
import os
import pandas as pd

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import Document
from llama_index.core.utils import set_global_tokenizer
from transformers import AutoTokenizer
from typing import Iterator
from pyspark.sql.functions import col, udf, length, pandas_udf, explode
from PyPDF2 import PdfReader

In [0]:
from mlflow.deployments import get_deploy_client


# gte-large-en Foundation models are available using the /serving-endpoints/databricks-gte-large-en/invocations api. 
deploy_client = get_deploy_client("databricks")


In [0]:
# Define a function to splot the text content into chuncks
@pandas_udf("array<string>")
def read_as_chunck(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    #Set llamma2 as tokenizer
    set_global_tokenizer(
        AutoTokenizer.from_pretrained("hf-internal-testing/llama-tokenizer")
    )
    # Sentence splitter from llama_index to split on sentences
    splitter = SentenceSplitter(chunk_size=500, chunck_overlap = 50)

    def extract_and_split(b):
        txt = parse_bytes_pypdf(b) 
        if txt is None: 
            return[]
        nodes = splitter.get_nodes_from_documentss([Document(text=txt)])
        return[n.text for n in nodes]
    for x in batch_iter:
        yield x.apply(extract_and_split)

In [0]:

# Set the Huggine Face cache directory to a writable location

df_chunks = (df
             .withColumn("content", explode(array(read_as_chunk("content"))))
             .selectExpr("path as pdf_name", "content")
             )
display(df_chunks)



In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.default.pdf_text_embeddings (
  id BIGINT GENERATED BY DEFAULT AS IDENTITY,
  pdf_name STRING,
  content STRING,
  embedding ARRAY <FLOAT>
  -- NOTE: the table has to be CDC because VectorSearch is using DLT that is requiring CDC state
  ) TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [0]:
@pandas_udf("array<float>")
def get_embedding(contents: pd.Series) -> pd.Series:
    import mlflow.deployments
    deploy_client = mlflow.deployments.get_deploy_client("databricks")
    def get_embeddings(batch):
        # NOTE: this will fail if an exception is thrown during embedding creation (add try/except if needed) 
        response = deploy_client.predict(endpoint="databricks-gte-large-en", inputs={"input": batch})
        return [e["embedding"] for e in response.data]

    # splitting the contents into batches of 150 items each, since the embedding model takes at most 150 inputs per request.
    max_batch_size = 150
    batches = [contents.iloc[i:i + max_batch_size] for i in range(0, len(contents), max_batch_size)]

    # process each batch and collect the results
    all_embeddings = []
    for batch in batches:
        all_embeddings += get_embeddings(batch.tolist())

    return pd.Series(all_embeddings)

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
import pyspark.sql.functions as F

df_chunk_emd = (df_chunks
                .withColumn("embedding", get_embedding("content"))
                .selectExpr("pdf_name", "content", "embedding")
                )
#display(df_chunk_emd)